Загрузка библиотек и данных

In [1]:
!pip install simpletransformers==0.61.13
!pip uninstall transformers
!pip install transformers==4.10.0
!pip install wikipedia
!pip install googledrivedownloader

     |████████████████████████████████| 221 kB 5.6 MB/s 
     |████████████████████████████████| 3.3 MB 29.3 MB/s 
     |████████████████████████████████| 1.7 MB 36.6 MB/s 
     |████████████████████████████████| 1.2 MB 46.1 MB/s 
     |████████████████████████████████| 287 kB 47.9 MB/s 
     |████████████████████████████████| 8.3 MB 32.2 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 2.9 MB 33.1 MB/s 
     |████████████████████████████████| 124 kB 46.0 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 895 kB 30.6 MB/s 
     |████████████████████████████████| 596 kB 26.1 MB/s 
     |████████████████████████████████| 97 kB 6.5 MB/s 
     |████████████████████████████████| 139 kB 43.2 MB/s 
     |████████████████████████████████| 180 kB 49.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 125 kB 43.3 MB/s 
     |█████████████████

Found existing installation: transformers 4.11.3
Uninstalling transformers-4.11.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.7/dist-packages/transformers-4.11.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/transformers/*
Proceed (y/n)? y
  Successfully uninstalled transformers-4.11.3
     |████████████████████████████████| 2.8 MB 5.2 MB/s 
Cloning into 'WikiSearch'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 93 (delta 32), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (93/93), done.


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('wordnet')
stopwords = stopwords.words("english")
lemmatizer = nltk.stem.WordNetLemmatizer() 

from nltk.tokenize import RegexpTokenizer, word_tokenize, sent_tokenize
main_tokenizer = RegexpTokenizer(r'\w+',)
sec_tokenizer = RegexpTokenizer(r'\S+')
 
from sklearn.externals import joblib
import numpy as np
from google.colab import output
import urllib
import difflib
import wikipedia
import pandas as pd
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='13Nuwm7BV-4RXI9JqjPTDE9rcdupkKqlF',
                                    dest_path='/Data/AIIJC/aiijc_1578_goodFromTrain_pretrained.model')
gdd.download_file_from_google_drive(file_id='1V4bUkfLuno6tIWfUqFusg8pnQWRTHYxn',
                                    dest_path='/Data/AIIJC/newTest.csv')

Функции для предобработки текста

In [4]:
def normal_form(word): #Получение нормальной формы слова
  word = word.lower()
  return word
 
def clean_html(html): #Очистка html
  soup = BeautifulSoup(BeautifulSoup(html, "lxml").text)
  return str(soup.body)

def get_good_tokens(text): #Выделение ключевых слов
  good_tokens = []

  for tokens in tokenizer(text)[1]:
    for token in tokens:
      token = normal_form(token)
      if token not in stopwords:
        good_tokens.append(token)
  return good_tokens

def tokenizer(text): #Токенизация текста в обработанные и необработанные токены
  raw_tokens = sec_tokenizer.tokenize(text)
  clean_tokens = main_tokenizer.tokenize_sents(raw_tokens)
  
  nClean_tokens = []
  for i in range(len(clean_tokens)):
    nClean_tokens.append([])
    for m in range(len(clean_tokens[i])):
      if normal_form(clean_tokens[i][m]) != 's':
        nClean_tokens[i].append(normal_form(clean_tokens[i][m]))

  return (raw_tokens, nClean_tokens)
 
def similarity(s1, s2): #Нахождение коэффициента схожести между двумя строками
  normalized1 = s1.lower()
  normalized2 = s2.lower()
  matcher = difflib.SequenceMatcher(None, normalized1, normalized2)
  return matcher.ratio()

def part_extractor(data,question,step,part_length): #Функция выделения релевантного фрагмента (Текст, вопрос, длинна фрагмента)
  good_tokens = get_good_tokens(question)
  
  tokens = tokenizer(data)
  
    
  for i in range(step-(len(tokens[0]) % step)): #Увеличение количества токенов до кратного длины части
    tokens[0].append('')
    tokens[1].append('')
 

  match_counter = 0 #Счетчик точных совпадений токенов
  best_part = '' #Лучшая часть
  max_match_qty = 0 #Максимальное количество совпавших токенов
 
  main_clrTokens = tokens[1]
  main_tokens = tokens[0]
  
  for i in range(0,len(tokens[0])-1,part_length): #Нахождение наиболее релевантной части текста
    tokens = main_tokens[i:i+part_length-1]
    clrTokens = main_clrTokens[i:i+part_length-1]

    for good_token in good_tokens:
      if in_tokens(good_token,clrTokens):
        match_counter += 1

    if match_counter > max_match_qty:
      max_match_qty = match_counter
      best_part = tokens
      
   
    match_counter = 0
  
  

  fin = '' #Восстановление текста
  for i in best_part:
    fin += (i+' ')

  
  return fin

def in_tokens(token,text):
  for i in text:
    for m in i:
      if token == m:
        return True
  return False

Загрузка данных и модели

In [5]:
test = pd.read_csv("/Data/AIIJC/newTest.csv",index_col=0, error_bad_lines=False,sep=",")
test.predict_answer = ''
test

,question,link_1,link_2,predict_answer
id,,,,
4001,When was the Dewey County Courthouse built?,https://en.wikipedia.org/wiki/Dewey_County_Cou...,"https://en.wikipedia.org/wiki/Dewey_County,_Ok...",
4002,What was the candle company founded in 1899?,https://en.wikipedia.org/wiki/S._S._Kresge,https://en.wikipedia.org/wiki/Bell_Telephone_C...,
4003,What is the purpose of the toroidal expansion ...,https://en.wikipedia.org/wiki/Toroidal_expansi...,https://en.wikipedia.org/wiki/Expansion_joint,
4004,What was the name of the route it was intended...,https://en.wikipedia.org/wiki/Pulaski_Skyway,https://en.wikipedia.org/wiki/Garden_State_Par...,
4005,What was the main focus of the republican move...,https://en.wikipedia.org/wiki/Republican_Party...,https://en.wikipedia.org/wiki/Tea_Party_movement,
...,...,...,...,...
6459,How many times has Michael Harry been a Danish...,https://en.wikipedia.org/wiki/Kasper_Schmeichel,https://en.wikipedia.org/wiki/George_Michael,
6460,Who was the earliest to propose the use of gue...,https://en.wikipedia.org/wiki/Guerrilla_warfare,https://en.wikipedia.org/wiki/History_of_guerr...,
6461,Howard Anthony Gayle was born?,https://en.wikipedia.org/wiki/Howard_Gayle,https://en.wikipedia.org/wiki/Marcus_Gayle,


In [6]:
model = joblib.load('/Data/AIIJC/aiijc_1578_goodFromTrain_pretrained.model')

model.args.max_seq_length = 512
model.args.silent = True

Просчет ответов для тестовой выборки

In [9]:
for i in range(len(test)): #Перебираем все вопросы
  question = test.question.iloc[i]

  try:
    link_1 = test.link_1.iloc[i].replace('https://en.wikipedia.org/wiki/','') #Убераем начало ссылки
    link_1 = urllib.parse.unquote(link_1) #Заменяем кривые символы на оригинал
    data_1 = wikipedia.page(link_1,auto_suggest=False).content #Парсим страничку вики
    data_1 = data_1.replace('\n',' ')
  except:
    pass
  try:
    link_2 = test.link_2.iloc[i].replace('https://en.wikipedia.org/wiki/','') #Убераем начало ссылки
    link_2 = urllib.parse.unquote(link_2) #Заменяем кривые символы на оригинал
    data_2 = wikipedia.page(link_2,auto_suggest=False).content #Парсим страничку вики
    data_2 = data_2.replace('\n',' ')
  except:
    pass


  try:
    context = part_extractor(data_1,question,16,64) #Выделяем фрагмент из первой статьи
  except:
    pass
 
  try:
    context += ' ' + part_extractor(data_2,question,16,32) #Выделяем фрагмент из второй статьи
  except:
    pass


  predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
  
  #Если ответ не был найден то перебираем различные длины фрагментов

  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,64)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,64)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass

  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,128)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
  
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,128)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
 
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_1,question,16,256)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass
 
  if predict[0]['answer'][0] == 'empty':
    try:
      context = part_extractor(data_2,question,16,256)
      predict = model.predict([{'context': context,'qas': [{'id': 0, 'question': question}]}])[0]
    except:
      pass


  if (i % 5) == 0:
    output.clear()
 

  print(i)
 
  test.predict_answer.iloc[i] = predict[0]['answer'][0]

5
6
7
8
9


Сохранение результатов

In [ ]:
sample = pd.read_csv('sample_submission.csv')

In [ ]:
sample

,id,answer
0,4001,answer
1,4002,answer
2,4003,answer
3,4004,answer
4,4005,answer
...,...,...
2458,6459,answer
2459,6460,answer
2460,6461,answer
2461,6462,answer


In [ ]:
for i in range(len(sample)):
  sample.answer.iloc[i] = test.predict_answer.iloc[i]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
sample.to_csv('submission.csv')